In [ ]:
from epx import Job, ModelConfig, SynthPop

# For visualization
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import requests

# Use the Epistemix default plotly template
r = requests.get("https://gist.githubusercontent.com/daniel-epistemix/8009ad31ebfa96ac97b7be038c014c0d/raw/320c3b0ca3dfbf7946e49c97254fa65d4753aeac/epx_plotly_theme.json")
if r.status_code == 200:
    pio.templates["epistemix"] = go.layout.Template(r.json())
    pio.templates.default = "epistemix"


# Transaction Model

This FRED model has all agents 18 and older go shopping. The number of stores,  items available, and item prices are specified in `data/item_inventory_TRANSACTIONS.csv`. The job will print out receipts as agents shop at their chosen stores.

In [ ]:
import time
# Define model configuration. This replaces the simulation block in 
    # FRED version 11.
transaction_config = ModelConfig(synth_pop=SynthPop("US_2010.v5", ["Loving_County_TX"]),
    start_date = "2022-05-10",
    end_date = "2022-05-11")

results_dir = "/home/epx/cl-results"

# Configure FRED job
transaction_job = Job(
    "model/transactions.fred",
    config=[transaction_config],
    key="transaction_job",
    results_dir=results_dir,
    size="hot",
    # Select FRED version compatible with selected model
    fred_version="11.0.1"
)


# Execute job
transaction_job.execute()

# the following loop idles while we wait for the simulation job to finish and periodically prints an update
update_count = 0
update_interval = 3
start_time = time.time()
timeout   = 300 # timeout in seconds
idle_time = 20   # time to wait (in seconds) before checking status again
while str(transaction_job.status) != 'DONE':
    if str(transaction_job.status) == 'ERROR':
        logs = transaction_job.status.logs
        log_msg = "; ".join(logs.loc[logs.level == "ERROR"].message.tolist())
        print(f"Job failed with the following error:\n '{log_msg}'")
        break
    if time.time() > start_time + timeout:
        msg = f"Job did not finish within {timeout / 60} minutes."
        raise RuntimeError(msg)
    
    if update_count >= update_interval:
        update_count = 0
        print(f"Job is still processing after {time.time() - start_time:.0f} seconds")
        
    update_count += 1
    
    time.sleep(idle_time)

print(f"Job completed in {time.time() - start_time:.0f} seconds")

str(transaction_job.status)

## Data outputs

First, let's look at the CSV file that we generated: 

In [ ]:
transactions_df = transaction_job.results.csv_output('transactions.csv')
transactions_df

Let's use it to count how many people of each demographic group shopped at each store:

In [ ]:
transactions_df[['id','race','store']].groupby(['race','store']).count()

We can visualize how much was spent at each of the three stores:

In [ ]:
fig = px.histogram(
    transactions_df,
    x='store',
    y='total_spent',
    title="Total spent at each store"
)
fig.update_layout(
    yaxis_title="Total spent ($)",
    xaxis_title="Store number",
    bargap=0.1,
    title_font_size=30
)
fig.show()

And we can look at the shopping habits of each age group:

In [ ]:
fig = px.histogram(
    transactions_df,
    x='age',
    y='total_spent',
    color = 'store',
    title="Total spent by age group"
)
fig.update_layout(
    yaxis_title="Total spent ($)",
    xaxis_title="Age",
    legend_title = "Store No.",
    bargap=0.1,
    title_font_size=30
)
fig.show()

To conserve resources, be sure to delete your job and its associated results once the job itself is out-of-scope:

In [ ]:
transaction_job.delete(interactive=False)